In [12]:
:dep polars = "0.35.4"

use polars::prelude::*;

fn read_csv(file_name: &str) -> Result<DataFrame, Box<dyn std::error::Error>> {
    let file = std::fs::File::open(file_name);
    match file {
        Ok(file) => {
            let mut df = CsvReader::new(file)
                .infer_schema(Some(100))
                .has_header(false)
                .finish()?;
            df = df.rename("column_1", "time")?.clone()
                .rename("column_2", "scale_id")?.clone()
                .rename("column_3", "raw")?.clone()
                .rename("column_4", "_")?.clone()
                .rename("column_5", "weight")?.clone()
                .rename("column_6", "encoder")?.clone();
            Ok(df)
        },
        Err(e) => Err(Box::new(e)),
    }
}

In [13]:
let file_name = "2023.11.21__16_05_53.csv";

match read_csv(file_name) {
    Ok(df) => println!("{:?}", df),
    Err(e) => eprintln!("Error reading CSV: {}", e),
};

shape: (56_213, 6)
┌────────────────┬──────────┬────────┬─────┬────────┬─────────┐
│ time           ┆ scale_id ┆ raw    ┆ _   ┆ weight ┆ encoder │
│ ---            ┆ ---      ┆ ---    ┆ --- ┆ ---    ┆ ---     │
│ i64            ┆ i64      ┆ f64    ┆ str ┆ f64    ┆ f64     │
╞════════════════╪══════════╪════════╪═════╪════════╪═════════╡
│ 63836179553062 ┆ 3        ┆ 0.2595 ┆ B   ┆ 0.57   ┆ 318.2   │
│ 63836179553069 ┆ 2        ┆ 0.1062 ┆ A   ┆ 28.11  ┆ 318.2   │
│ 63836179553078 ┆ 4        ┆ 0.1499 ┆ B   ┆ 0.32   ┆ 318.3   │
│ 63836179553085 ┆ 1        ┆ 0.1724 ┆ A   ┆ 17.11  ┆ 318.4   │
│ …              ┆ …        ┆ …      ┆ …   ┆ …      ┆ …       │
│ 63836180002112 ┆ 3        ┆ 0.2026 ┆ B   ┆ 0.49   ┆ 240.5   │
│ 63836180002116 ┆ 1        ┆ 0.1637 ┆ A   ┆ 5.13   ┆ 240.5   │
│ 63836180002128 ┆ 4        ┆ 0.1138 ┆ B   ┆ 0.27   ┆ 240.6   │
│ 63836180002132 ┆ 2        ┆ 0.088  ┆ A   ┆ 3.59   ┆ 240.6   │
└────────────────┴──────────┴────────┴─────┴────────┴─────────┘


In [14]:
fn read_csv(file_name: &str) -> Result<DataFrame, Box<dyn std::error::Error>> {
    let file = std::fs::File::open(file_name);
    match file {
        Ok(file) => {
            let mut df = CsvReader::new(file)
                .infer_schema(Some(100))
                .has_header(false)
                .finish()?;
            df = df.rename("column_1", "time")?.clone()
                .rename("column_2", "scale_id")?.clone()
                .rename("column_3", "raw")?.clone()
                .rename("column_4", "_")?.clone()
                .rename("column_5", "weight")?.clone()
                .rename("column_6", "encoder")?.clone();
            let scale_id = df.column("scale_id")?.cast(&DataType::UInt32)?;
            df = df.with_column(scale_id)?.clone();
            let mut circle = 0;
            let mut last_encoder = 0.0;
            let circles: Vec<u32> = df.column("encoder")?.f64()?
                .into_iter()
                .map(|encoder| {
                    if let Some(encoder) = encoder {
                        if encoder < last_encoder {
                            circle += 1;
                        }
                        last_encoder = encoder;
                    }
                    circle
                })
                .map(|x| x as u32)
                .collect();
            let circle_series = Series::new("circle", circles);
            df = df.with_column(circle_series)?.clone();
            Ok(df)
        },
        Err(e) => Err(Box::new(e)),
    }
}

In [15]:
let file_name = "2023.11.21__16_05_53.csv";

match read_csv(file_name) {
    Ok(df) => println!("{:?}", df),
    Err(e) => eprintln!("Error reading CSV: {}", e),
};

shape: (56_213, 7)
┌────────────────┬──────────┬────────┬─────┬────────┬─────────┬────────┐
│ time           ┆ scale_id ┆ raw    ┆ _   ┆ weight ┆ encoder ┆ circle │
│ ---            ┆ ---      ┆ ---    ┆ --- ┆ ---    ┆ ---     ┆ ---    │
│ i64            ┆ u32      ┆ f64    ┆ str ┆ f64    ┆ f64     ┆ u32    │
╞════════════════╪══════════╪════════╪═════╪════════╪═════════╪════════╡
│ 63836179553062 ┆ 3        ┆ 0.2595 ┆ B   ┆ 0.57   ┆ 318.2   ┆ 0      │
│ 63836179553069 ┆ 2        ┆ 0.1062 ┆ A   ┆ 28.11  ┆ 318.2   ┆ 0      │
│ 63836179553078 ┆ 4        ┆ 0.1499 ┆ B   ┆ 0.32   ┆ 318.3   ┆ 0      │
│ 63836179553085 ┆ 1        ┆ 0.1724 ┆ A   ┆ 17.11  ┆ 318.4   ┆ 0      │
│ …              ┆ …        ┆ …      ┆ …   ┆ …      ┆ …       ┆ …      │
│ 63836180002112 ┆ 3        ┆ 0.2026 ┆ B   ┆ 0.49   ┆ 240.5   ┆ 11     │
│ 63836180002116 ┆ 1        ┆ 0.1637 ┆ A   ┆ 5.13   ┆ 240.5   ┆ 11     │
│ 63836180002128 ┆ 4        ┆ 0.1138 ┆ B   ┆ 0.27   ┆ 240.6   ┆ 11     │
│ 63836180002132 ┆ 2        ┆ 0.